In [1]:
!pip install --extra-index-url=https://azuremlsdktestpypi.azureedge.net/CLI-SDK-Runners-Validation/14249619 azureml-defaults==0.1.0.14249619 azureml-pipeline-wrapper==0.1.0.14249619 azureml-designer-core --user --upgrade

/lib/python3.7/site-packages (from azureml-defaults==0.1.0.14249619) (0.2)
  Using cached jsonschema-3.0.1-py2.py3-none-any.whl (54 kB)
  Using cached pycryptodomex-3.7.3-cp37-cp37m-manylinux1_x86_64.whl (7.5 MB)
  Using cached more_itertools-6.0.0-py3-none-any.whl (52 kB)
ERROR: pytest-astropy 0.8.0 requires pytest-cov>=2.0, which is not installed.
ERROR: pytest-astropy 0.8.0 requires pytest-filter-subpackage>=0.1, which is not installed.
ERROR: azureml-designer-core 0.0.35 has requirement ruamel.yaml==0.15.89, but you'll have ruamel-yaml 0.15.87 which is incompatible.


In [3]:
!python --version

Python 3.6.10 :: Anaconda, Inc.


In [1]:
from azureml.core import Workspace, Run, Dataset
from azureml.pipeline.wrapper import Pipeline, dsl, Module

ws = Workspace.get(name='kubeflow_ws_1', subscription_id='74eccef0-4b8d-4f83-b5f9-fa100d155b22', resource_group='kubeflow-demo')

train_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='MPI Train')
score_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Score')
eval_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Evaluate')
compare_module_func = Module.load(ws, namespace='microsoft.com/aml/samples', name='Compare 2 Models')

train_data = Dataset.get_by_name(ws, 'training_data')
test_data = Dataset.get_by_name(ws, 'test_data')

In [3]:
# define a sub pipeline
@dsl.pipeline(name = 'Train-Score-Eval', 
              description = 'train model and evaluate model perf')
def training_pipeline(input_data, learning_rate):
    train = train_module_func(
        training_data=input_data, 
        max_epochs=5, 
        learning_rate=learning_rate)
   
    train.runsettings.configure(process_count_per_node = 2, node_count = 2)

    score = score_module_func(
        model_input=train.outputs.model_output, 
        test_data=test_data)

    eval = eval_module_func(scoring_result=score.outputs.score_output)
    
    return {'eval_output': eval.outputs.eval_output, 'model_output': train.outputs.model_output}

In [5]:
# define pipeline with sub pipeline
@dsl.pipeline(name = 'Dummy automl pipeline', 
              description = 'select best model trained with different learning rate',
              default_compute_target = 'cpu-cluster')
def dummy_automl_pipeline():
    train_and_evalute_model1 = training_pipeline(train_data, 0.01)
    train_and_evalute_model2 = training_pipeline(train_data, 0.02)
    
    compare = compare_module_func(
        model1=train_and_evalute_model1.outputs.model_output, 
        eval_result1=train_and_evalute_model1.outputs.eval_output,
        model2=train_and_evalute_model2.outputs.model_output,
        eval_result2=train_and_evalute_model2.outputs.eval_output
    )

    return {**compare.outputs}

# create a pipeline
pipeline = dummy_automl_pipeline()

In [6]:
pipeline.validate()

{'result': 'validation passed', 'errors': []}

In [7]:
pipeline.submit(experiment_name='samples')

Submitted PipelineRun f33b0eed-5100-4ced-9b3f-8aa3c6c4f061
Link to Azure Machine Learning Portal: https://ml.azure.com/experiments/samples/runs/f33b0eed-5100-4ced-9b3f-8aa3c6c4f061?wsid=/subscriptions/74eccef0-4b8d-4f83-b5f9-fa100d155b22/resourcegroups/kubeflow-demo/workspaces/kubeflow_ws_1


Experiment,Id,Type,Status,Details Page,Docs Page
samples,f33b0eed-5100-4ced-9b3f-8aa3c6c4f061,azureml.PipelineRun,NotStarted,Link to Azure Machine Learning studio,Link to Documentation
